In [1]:
from modules import data_collection, models, optimizer, data_cleaning
import pandas as pd
import _pickle as cPickle

In [2]:
data_20 = pd.read_csv('../Data/FPL_20.csv')
data_21 = pd.read_csv('../Data/FPL_21.csv')
data_22 = pd.read_csv('../Data/FPL_22.csv')
with open(r"../Data/FBREF_22_23.pickle", "rb") as inputfile:
    df = cPickle.load(inputfile)

In [3]:
search, y_pred, mse, rmse, r2 = models.model(df.drop(["Player","Cost"], axis = 1))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


In [4]:
for i in [mse, rmse, r2]:
    print(i)

274.5887225594264
16.570718830498162
0.44440420707976236


In [61]:
expected_scores = search.best_estimator_.predict(df.drop(["Player","Cost"], axis = 1))
prices = (df.Cost / 10).to_list()
position = df.Position.to_list()
team = df.Squad.to_list()
names = df.Player.to_list()

AttributeError: 'list' object has no attribute 'map'

In [71]:
import importlib as imp

imp.reload(optimizer)

decisions, captain_decisions, sub_decisions = optimizer.select_team(
    expected_scores, prices, position, team
)

325
6.4
4.5
5.0
4.7
7.2
5.3
5.3
5.3
4.4
4.3
4.5
7.0
4.2
4.3
5.5
4.7
4.5
3.9
8.7
5.3
4.5
4.8
4.5
4.0
7.2
4.5
4.9
6.9
4.5
4.5
4.2
5.1
5.1
4.0
5.6
5.4
5.5
5.5
3.9
4.5
4.7
4.2
8.0
4.0
5.3
3.9
4.5
4.5
4.4
3.9
7.9
4.4
7.4
4.6
4.7
4.7
4.2
4.2
5.8
4.5
4.3
4.5
4.5
5.0
4.4
4.4
5.9
4.8
4.4
5.7
4.9
3.9
4.2
4.0
4.9
5.4
4.8
12.6
4.4
4.7
5.8
4.9
4.3
7.8
5.2
4.6
4.6
4.3
5.0
4.6
5.3
4.0
4.3
3.9
4.7
4.7
5.0
4.2
5.2
6.3
4.4
5.7
4.5
3.9
4.4
8.1
8.3
4.3
4.4
4.5
3.9
5.2
4.4
3.9
5.7
4.3
4.5
4.5
5.1
4.8
4.8
4.8
5.5
4.3
4.3
3.9
5.3
5.3
6.8
4.2
5.6
4.5
4.4
5.8
4.4
7.7
5.7
5.3
4.5
3.9
4.5
11.6
4.2
4.3
5.3
4.8
6.1
6.6
4.5
5.6
6.0
6.0
5.8
5.5
4.2
4.8
6.8
4.4
4.9
3.9
4.3
11.6
4.8
4.2
4.4
3.9
4.6
4.5
4.8
4.5
4.3
8.0
4.9
4.2
5.1
5.8
4.3
4.5
4.4
4.3
5.4
4.4
5.5
5.0
4.4
5.4
8.2
4.7
7.5
4.8
5.0
6.7
4.5
5.2
6.1
5.8
4.8
4.3
5.1
5.2
4.9
4.6
4.8
4.3
4.5
4.4
4.3
4.3
4.5
4.4
7.5
4.3
5.3
4.3
4.8
4.8
6.5
5.3
4.4
5.4
3.9
4.8
4.9
4.3
4.8
5.3
4.7
4.5
4.4
4.8
4.5
4.4
5.3
5.7
4.9
5.2
6.7
5.3
4.2
4.8
4.5
4.2
5.0
6.8
4.0
4.9
4.9
6.2
8

TypeError: A False object cannot be passed as a constraint

In [29]:
pos_map = {1:'GK', 2:'DEF', 3: 'MID', 4:'FWD'}
team_names = []
for i in range(len(decisions)):
    if decisions[i].value() != 0:
        team_names.append(names[i])
        print(f'{names[i]} {pos_map[position[i]]}: {expected_scores[i]} points at ${prices[i]}')
print('subs')
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() != 0:
        team_names.append(names[i])
        print(f'{names[i]} {pos_map[position[i]]}: {expected_scores[i]} points at ${prices[i]}')
#print('Capt')
#for i in range(test_data.shape[0]):
#    if captain_decisions[i].value != 0:
#        print(f'Captain: {names[i]} score: {captain_decisions[i].value}')

NameError: name 'decisions' is not defined

In [ ]:
print(test_data[test_data.second_name == "Kane"].FPL_points)
print(y_pred[test_data[test_data.second_name == "Kane"].index])

In [ ]:
test_data[test_data.second_name.isin(team_names)].FPL_points